In [99]:
import random
from sympy import isprime, mod_inverse
from math import gcd
import clipboard

In [100]:
def modinv(a, p):
    return pow(a, -1, p)

In [101]:
BITS = 20

In [102]:
class ElGamal:
    def __init__(self, bits=BITS):
        """Инициализация системы Эль-Гамаля"""
        self.p = self.generate_large_prime(bits)
        self.g = self.find_primitive_root(self.p)
        # self.public_key: p, g, y
        self.public_key, self.private_key = self.generate_keys()
        
    def generate_large_prime(self, bits):
        """Генерация большого простого числа"""
        while True:
            num = random.getrandbits(bits)
            if num % 2 != 0 and isprime(num):
                return num
    
    def find_primitive_root(self, p):
        """Поиск первообразного корня по модулю p"""
        if p == 2:
            return 1
        
        # Факторизация p-1
        phi = p - 1
        factors = self.factorize(phi)
        
        # Поиск первообразного корня
        for g in range(2, p):
            if all(pow(g, phi // factor, p) != 1 for factor in factors):
                return g
        return None
    
    def factorize(self, n):
        """Факторизация числа на простые множители"""
        factors = set()
        d = 2
        while d * d <= n:
            while n % d == 0:
                factors.add(d)
                n //= d
            d += 1 if d == 2 else 2
        if n > 1:
            factors.add(n)
        return factors
    
    def generate_keys(self):
        """Генерация пары ключей"""
        # Закрытый ключ
        x = random.randint(2, self.p - 2)
        # Открытый ключ
        y = pow(self.g, x, self.p)
        return (self.p, self.g, y), x  # (открытый ключ), закрытый ключ
    
    def encrypt_char(self, m):
        """Шифрование одного символа"""
        p, g, y = self.public_key
        
        # Преобразуем символ в число
        m_num = ord(m)
        if m_num >= p:
            raise Exception("TOO LOW P, RAISE BITS")
        
        # Выбираем случайное k
        k = random.randint(2, p - 2)
        
        # Вычисляем компоненты шифротекста
        a = pow(g, k, p)
        b = (pow(y, k, p) * m_num) % p
        
        return (a, b)
    
    def encrypt(self, message):
        """Шифрование сообщения"""
        encrypted = []
        for char in message:
            encrypted.append(self.encrypt_char(char))
        return encrypted
    
    def decrypt_char(self, cipher_pair):
        """Дешифрование одного символа"""
        p, g, y = self.public_key  # для доступа к p
        x = self.private_key
        a, b = cipher_pair
        
        # Вычисляем s = a^x mod p
        s = pow(a, x, p)
        
        # Вычисляем обратный элемент s_inv
        s_inv = mod_inverse(s, p)
        
        # Восстанавливаем исходное сообщение
        m = (b * s_inv) % p
        return chr(m)
    
    def decrypt(self, ciphertext):
        """Дешифрование сообщения"""
        decrypted = []
        for a, b in ciphertext:
            decrypted.append(self.decrypt_char((a, b)))
        return ''.join(decrypted)




In [103]:

elgamal = ElGamal()
# elgamal.public_key = (11, 2, 6)
# elgamal.private_key = 9
# Генерация ключей

print(f"Открытый ключ (p, g, y): {elgamal.public_key}")
print(f"Закрытый ключ (x): {elgamal.private_key}")

# Исходное сообщение
message = "Hello, ElGamal!"
print(f"Исходное сообщение: {message}")

# Шифрование
ciphertext = elgamal.encrypt(message)
print(f"Шифротекст (пары a, b): {ciphertext}")
clipboard.copy(str(ciphertext))
print("скопировано")
for i, (a, b) in enumerate(ciphertext):
    print(f"  Символ {i}: a={a}, b={b}")

# Дешифрование
decrypted_message = elgamal.decrypt(ciphertext)
print(f"Дешифрованное сообщение: {decrypted_message}")

# Проверка
print(f"Сообщение восстановлено корректно: {message == decrypted_message}")


Открытый ключ (p, g, y): (309737, 3, 267632)
Закрытый ключ (x): 229685
Исходное сообщение: Hello, ElGamal!
Шифротекст (пары a, b): [(120859, 78837), (137244, 308861), (296511, 136888), (171724, 149503), (261467, 123159), (118867, 196302), (28955, 121801), (25112, 21846), (5856, 30065), (211047, 24745), (92584, 51509), (196945, 305586), (276889, 46080), (200997, 180680), (110210, 230943)]
скопировано
  Символ 0: a=120859, b=78837
  Символ 1: a=137244, b=308861
  Символ 2: a=296511, b=136888
  Символ 3: a=171724, b=149503
  Символ 4: a=261467, b=123159
  Символ 5: a=118867, b=196302
  Символ 6: a=28955, b=121801
  Символ 7: a=25112, b=21846
  Символ 8: a=5856, b=30065
  Символ 9: a=211047, b=24745
  Символ 10: a=92584, b=51509
  Символ 11: a=196945, b=305586
  Символ 12: a=276889, b=46080
  Символ 13: a=200997, b=180680
  Символ 14: a=110210, b=230943
Дешифрованное сообщение: Hello, ElGamal!
Сообщение восстановлено корректно: True


In [104]:
class ElGamalCryptanalysis:
    def __init__(self, public_key):
        self.p, self.g, self.y = public_key
    
    def brute_force_dlp(self, max_trials=1000000):
        """Атака методом полного перебора на задачу дискретного логарифма"""
        print("Начало атаки методом полного перебора...")
        
        for x in range(2, min(max_trials, self.p)):
            if pow(self.g, x, self.p) == self.y:
                print(f"Найден закрытый ключ x = {x}")
                return x
        
        print(f"Закрытый ключ не найден за {max_trials} попыток")
        return None
    
    def baby_step_giant_step(self):
        """Алгоритм Гельфонда-Шенкса (baby-step giant-step)"""
        print("Применение алгоритма Гельфонда-Шенкса...")
        
        m = int(self.p ** 0.5) + 1
        
        # Baby steps
        baby_steps = {}
        for j in range(m):
            value = pow(self.g, j, self.p)
            baby_steps[value] = j
        
        # Вычисление g^{-m} mod p
        gm_inv = pow(pow(self.g, m, self.p), self.p - 2, self.p)
        
        # Giant steps
        current = self.y
        for i in range(m):
            if current in baby_steps:
                x = i * m + baby_steps[current]
                print(f"Найден закрытый ключ x = {x}")
                return x
            current = (current * gm_inv) % self.p
        
        print("Закрытый ключ не найден")
        return None


In [105]:
def analyze(cipher: list[tuple[int, int]], public_pgy: tuple[int, int, int]) -> None:
    # Создаем систему Эль-Гамаля
    elgamal = ElGamal()
    elgamal.private_key = None
    elgamal.public_key = public_pgy

    print("шифр", cipher)

    print("=" * 60)
    print("КРИПТОАНАЛИЗ СИСТЕМЫ ЭЛЬ-ГАМАЛЯ")
    print("=" * 60)
    print(f"Открытый ключ: p={elgamal.public_key[0]}, g={elgamal.public_key[1]}, y={elgamal.public_key[2]}")

    cryptanalyst = ElGamalCryptanalysis(elgamal.public_key)

    # 1. Атака методом полного перебора
    print("" + "=" * 60)
    print("1. АТАКА МЕТОДОМ ПОЛНОГО ПЕРЕБОРА")
    print("=" * 60)
    found_key = cryptanalyst.brute_force_dlp(max_trials=10000000)
    if not found_key:
        print("НЕ ПОЛУЧИЛОСЬ")
    else:
        elgamal.private_key = found_key
        print(f"РАСШИФРОВАНО {elgamal.decrypt(cipher)}")


    # 2. Алгоритм Гельфонда-Шенкса
    print("" + "=" * 60)
    print("2. АЛГОРИТМ ГЕЛЬФОНДА-ШЕНКСА")
    print("=" * 60)
    found_key_bsgs = cryptanalyst.baby_step_giant_step()
    if not found_key_bsgs:
        print("НЕ ПОЛУЧИЛОСЬ")
    else:
        elgamal.private_key = found_key_bsgs
        print(f"РАСШИФРОВАНО {elgamal.decrypt(cipher)}")


In [106]:
cipher = [(629674, 629705), (185693, 233232), (613084, 687374), (6439, 452739), (699607, 313387), (272177, 541654), (509819, 819869), (223886, 340787), (813450, 332174), (180927, 586389), (94175, 141280), (398119, 489353), (748703, 683045), (576722, 418501), (787874, 59633), (190683, 229692), (650688, 456639), (134947, 542816), (433051, 810991), (409213, 624268), (265438, 615957)]

analyze(
    cipher=cipher,
    public_pgy=[848429, 2, 828709]
)

шифр [(629674, 629705), (185693, 233232), (613084, 687374), (6439, 452739), (699607, 313387), (272177, 541654), (509819, 819869), (223886, 340787), (813450, 332174), (180927, 586389), (94175, 141280), (398119, 489353), (748703, 683045), (576722, 418501), (787874, 59633), (190683, 229692), (650688, 456639), (134947, 542816), (433051, 810991), (409213, 624268), (265438, 615957)]
КРИПТОАНАЛИЗ СИСТЕМЫ ЭЛЬ-ГАМАЛЯ
Открытый ключ: p=848429, g=2, y=828709
1. АТАКА МЕТОДОМ ПОЛНОГО ПЕРЕБОРА
Начало атаки методом полного перебора...
Найден закрытый ключ x = 423532
РАСШИФРОВАНО Lol, Lmao, ahahahahha
2. АЛГОРИТМ ГЕЛЬФОНДА-ШЕНКСА
Применение алгоритма Гельфонда-Шенкса...
Найден закрытый ключ x = 423532
РАСШИФРОВАНО Lol, Lmao, ahahahahha


In [107]:
# def pollards_rho_dlog(g, y, p):
#     def f(x, a, b):
#         if x % 3 == 0:
#             return (x * g) % p, (a + 1) % (p - 1), b
#         elif x % 3 == 1:
#             return (x * y) % p, a, (b + 1) % (p - 1)
#         else:
#             return (x * x) % p, (2 * a) % (p - 1), (2 * b) % (p - 1)

#     x, a, b = 1, 0, 0
#     X, A, B = 1, 0, 0

#     while True:
#         x, a, b = f(x, a, b)
#         X, A, B = f(*f(X, A, B))

#         if x == X:
#             r = (a - A) % (p - 1)
#             s = (B - b) % (p - 1)

#             if gcd(s, p - 1) != 1:
#                 return None

#             return (r * modinv(s, p - 1)) % (p - 1)


In [108]:
# def mod_inverse(a, m):
#     def egcd(a, b):
#         if a == 0:
#             return b, 0, 1
#         g, y, x = egcd(b % a, a)
#         return g, x - (b // a) * y, y
#     g, x, _ = egcd(a, m)
#     if g != 1:
#         raise ValueError("Нет инверсии")
#     return x % m

# def pollard_rho_dl(g, y, p, max_steps=10**6, max_tries=20):
#     order = p - 1
#     for try_num in range(max_tries):
#         # Рандомизируем партицию (20 случайных коэффициентов для 20 "зон")
#         coeffs = [(random.randint(1, order-1), random.randint(0, order-1)) for _ in range(20)]
        
#         Xt = Xh = random.randint(1, p-1)
#         at = ah = random.randint(0, order-1)
#         bt = bh = random.randint(0, order-1)
        
#         for _ in range(max_steps):
#             # Шаг черепахи
#             zone_t = hash(Xt) % 20
#             Xt = (Xt * pow(g, coeffs[zone_t][0], p) * pow(y, coeffs[zone_t][1], p)) % p
#             at = (at + coeffs[zone_t][0]) % order
#             bt = (bt + coeffs[zone_t][1]) % order
            
#             # Шаг зайца (два шага)
#             for _ in range(2):
#                 zone_h = hash(Xh) % 20
#                 Xh = (Xh * pow(g, coeffs[zone_h][0], p) * pow(y, coeffs[zone_h][1], p)) % p
#                 ah = (ah + coeffs[zone_h][0]) % order
#                 bh = (bh + coeffs[zone_h][1]) % order
            
#             if Xt == Xh:
#                 denominator = (bt - bh) % order
#                 if denominator == 0:
#                     break  # плохое столкновение, новая попытка
#                 log = ((ah - at) * mod_inverse(denominator, order)) % order
#                 if pow(g, log, p) == y:
#                     return log
#                 # Иногда ложное совпадение X, но неверный лог — редкость, но продолжаем
#                 break
#     return None

In [109]:
# print("=" * 80)
# print("ПОЛНАЯ ДЕМОНСТРАЦИЯ ШИФРОВАНИЯ И КРИПТОАНАЛИЗА ЭЛЬ-ГАМАЛЯ")
# print("=" * 80)

# # Инициализация с небольшим модулем для демонстрации
# elgamal = ElGamal()

# real_private_key = elgamal.private_key
# elgamal.private_key = None

# test_text = "QWERTY!"
# cipher = elgamal.encrypt(test_text)

# p, g, y = elgamal.public_key

# print("Параметры системы:")
# print(f"p = {p}")
# print(f"g = {g}")
# print(f"y = {y}")
# print(f"Истинный закрытый ключ x = {real_private_key}")

# # Тестирование ρ-алгоритма Полларда
# # pollard_decryptor = PollardRhoDLP(p, g, y)
# # pollard_result = pollard_decryptor.pollard_rho(max_iter=100000)
# pollard_result = pollard_rho_dl(p, g, y)
# print(pollard_result, "pollard result")

# if pollard_result and pollard_result == real_private_key:
#     print("✓ ρ-алгоритм Полларда успешно нашел правильный ключ!")
#     elgamal.private_key = pollard_result
#     print(elgamal.decrypt(cipher))
# elif pollard_result:
#     print(pow(g, pollard_result, p), y, pow(g, pollard_result, p) == y)
#     print(f"✗ Найден ключ {pollard_result}, но он не совпадает с истинным")
#     elgamal.private_key = pollard_result
#     print(elgamal.decrypt(cipher))
# else:
#     print("✗ ρ-алгоритм не смог найти ключ")


